# 4章
- 以下で実行するコードには確率的な処理が含まれていることがあり、コードの出力結果と本書に記載されている出力例が異なることがあります。

In [3]:
# 4-1
!pip install transformers==4.18.0 fugashi==1.1.0 ipadic==1.0.0

In [4]:
# 4-2
import torch
from transformers import BertJapaneseTokenizer, BertModel

In [5]:
# 4-3
model_name = 'tohoku-nlp/bert-base-japanese-whole-word-masking'
tokenizer = BertJapaneseTokenizer.from_pretrained(model_name)

Downloading:   0%|          | 0.00/252k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/120 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/479 [00:00<?, ?B/s]

In [6]:
# 4-4
tokenizer.tokenize('明日は自然言語処理の勉強をしよう。')

['明日', 'は', '自然', '言語', '処理', 'の', '勉強', 'を', 'しよ', 'う', '。']

In [7]:
# 4-5
tokenizer.tokenize('明日はマシンラーニングの勉強をしよう。')

['明日', 'は', 'マシン', '##ラー', '##ニング', 'の', '勉強', 'を', 'しよ', 'う', '。']

In [8]:
# 4-6
tokenizer.tokenize('機械学習を中国語にすると机器学习だ。')

['機械', '学習', 'を', '中国', '語', 'に', 'する', 'と', '机', '器', '学', '[UNK]', 'だ', '。']

In [9]:
# 4-7
input_ids = tokenizer.encode('明日は自然言語処理の勉強をしよう。')
print(input_ids)

[2, 11475, 9, 1757, 1882, 2762, 5, 8192, 11, 2132, 205, 8, 3]


In [10]:
# 4-8
tokenizer.convert_ids_to_tokens(input_ids)

['[CLS]', '明日', 'は', '自然', '言語', '処理', 'の', '勉強', 'を', 'しよ', 'う', '。', '[SEP]']

In [19]:
# 4-9
text = '明日の天気は晴れだ。'
encoding = tokenizer(
    text, max_length=12, padding='max_length', truncation=True
)
print('# encoding:')
print(encoding)

tokens = tokenizer.convert_ids_to_tokens(encoding['input_ids'])
print('# tokens:')
print(tokens)

# encoding:
{'input_ids': [2, 11475, 5, 11385, 9, 16577, 75, 8, 3, 0, 0, 0], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0]}
# tokens:
['[CLS]', '明日', 'の', '天気', 'は', '晴れ', 'だ', '。', '[SEP]', '[PAD]', '[PAD]', '[PAD]']


In [20]:
# 4-10
encoding = tokenizer(
    text, max_length=6, padding='max_length', truncation=True
)
tokens = tokenizer.convert_ids_to_tokens(encoding['input_ids'])
print(tokens)

['[CLS]', '明日', 'の', '天気', 'は', '[SEP]']


In [21]:
# 4-11
text_list = ['明日の天気は晴れだ。','パソコンが急に動かなくなった。']
tokenizer(
    text_list, max_length=10, padding='max_length', truncation=True
)

{'input_ids': [[2, 11475, 5, 11385, 9, 16577, 75, 8, 3, 0], [2, 6311, 14, 1132, 7, 16084, 332, 58, 10, 3]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 0], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}

In [22]:
# 4-12
tokenizer(text_list, padding='longest')

{'input_ids': [[2, 11475, 5, 11385, 9, 16577, 75, 8, 3, 0, 0], [2, 6311, 14, 1132, 7, 16084, 332, 58, 10, 8, 3]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}

In [23]:
# 4-13
tokenizer(
    text_list,
    max_length=10,
    padding='max_length',
    truncation=True,
    return_tensors='pt'
)

{'input_ids': tensor([[    2, 11475,     5, 11385,     9, 16577,    75,     8,     3,     0],
        [    2,  6311,    14,  1132,     7, 16084,   332,    58,    10,     3]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [24]:
# 4-14
# モデルのロード
model_name = 'tohoku-nlp/bert-base-japanese-whole-word-masking'
bert = BertModel.from_pretrained(model_name)

# BERTをGPUに載せる
bert = bert.cuda()

Downloading:   0%|          | 0.00/424M [00:00<?, ?B/s]

Some weights of the model checkpoint at tohoku-nlp/bert-base-japanese-whole-word-masking were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [25]:
# 4-15
print(bert.config)

BertConfig {
  "_name_or_path": "tohoku-nlp/bert-base-japanese-whole-word-masking",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "tokenizer_class": "BertJapaneseTokenizer",
  "transformers_version": "4.18.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 32000
}



In [26]:
# 4-16
text_list = [
    '明日は自然言語処理の勉強をしよう。',
    '明日はマシーンラーニングの勉強をしよう。'
]

# 文章の符号化
encoding = tokenizer(
    text_list,
    max_length=32,
    padding='max_length',
    truncation=True,
    return_tensors='pt'
)

# データをGPUに載せる
encoding = { k: v.cuda() for k, v in encoding.items() }

# BERTでの処理
output = bert(**encoding) # それぞれの入力は2次元のtorch.Tensor
last_hidden_state = output.last_hidden_state # 最終層の出力

In [27]:
# 4-17
output = bert(
    input_ids=encoding['input_ids'],
    attention_mask=encoding['attention_mask'],
    token_type_ids=encoding['token_type_ids']
)

In [28]:
# 4-18
print(last_hidden_state.size()) #テンソルのサイズ

torch.Size([2, 32, 768])


In [30]:
# 4-19
with torch.no_grad():
    output = bert(**encoding)
    last_hidden_state = output.last_hidden_state

In [31]:
# 4-20
last_hidden_state = last_hidden_state.cpu() # CPUにうつす。
last_hidden_state = last_hidden_state.numpy() # numpy.ndarrayに変換
last_hidden_state = last_hidden_state.tolist() # リストに変換